# Example script demonstrating the portfolio optimization and backtesting system.

This script shows how to:
1. Load data from CSV files
2. Create and configure optimization strategies
3. Run backtests
4. Compare strategy performance
5. Generate reports and visualizations

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from datetime import datetime
import tempfile
import os
from pathlib import Path

# Import our modules
from optfolio.data.loader import DataLoader
from optfolio.data.validator import DataValidator
from optfolio.backtesting.engine import Backtester
from optfolio.strategies.base import StrategyFactory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_dir = "../data/price"

# Step 2: Load and validate data
print("\n1. Loading and validating data...")
data_loader = DataLoader(data_dir)
sectors = [
        "XLE", "XLF", "XLU", "XLI", "GDX", "XLK", "XLV", "XLY", "XLP", "XLB",
        "XOP", "IYR", "XHB", "ITB", "VNQ", "GDXJ", "IYE", "OIH", "XME", "XRT",
        "SMH", "IBB", "KBE", "KRE", "XTL"
    ]
sectors = ['WSM', 'PAYX', 'BMI', 'BK', 'NDAQ', 'MSI', 'WMT', 'TJX', 'AIG', 'RJF', 'V', 'CTAS', 'TT', 'TRGP', 'JPM', 'GE', 'MCK', 'PH', 'LLY', 'COST', 'AVGO', 'NEE', 'AMAT', 'ADI', 'SHW', 'INTU', 'KLAC']
# Load all available data
prices = data_loader.load_prices(tickers=sectors)
returns = data_loader.get_returns()

print(f"   Loaded {len(prices.columns)} tickers")
print(prices.columns)
print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
print(f"   Total observations: {len(prices)}")

# Validate data
validator = DataValidator()
price_issues = validator.validate_price_data(prices)
returns_issues = validator.validate_returns_data(returns)

print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")

if price_issues['warnings']:
    print(f"   Warnings: {len(price_issues['warnings'])}")
if price_issues['errors']:
    print(f"   Errors: {len(price_issues['errors'])}")


1. Loading and validating data...
   Loaded 27 tickers
Index(['AMAT', 'GE', 'AIG', 'PH', 'MCK', 'LLY', 'CTAS', 'RJF', 'WMT', 'PAYX',
       'BK', 'TT', 'AVGO', 'BMI', 'NEE', 'WSM', 'MSI', 'TJX', 'TRGP', 'ADI',
       'V', 'JPM', 'INTU', 'NDAQ', 'KLAC', 'SHW', 'COST'],
      dtype='object')
   Date range: 2015-09-02 00:00:00 to 2025-09-01 00:00:00
   Total observations: 2513
   Data quality score: 1.000
   Warnings: 1


In [4]:
print("\n2. Creating optimization strategies...")

strategies = [
    StrategyFactory.create('equal_weight'),
    StrategyFactory.create('mean_variance', objective="sortino_ratio"),
    StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
    StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
    StrategyFactory.create('black_litterman', 
                                    prior_method="market_cap",  # Use market cap weighted priors
                                    view_method="upside") 
]

strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
for i, strategy in enumerate(strategies):
    strategy.name = strategy_names[i]
    print(f"   Created: {strategy}")


2. Creating optimization strategies...
   Created: Equal Weight Strategy (Equal Weight)
   Created: Mean-Variance (Sortino) Strategy (sortino_ratio)
   Created: Mean-Variance (Sharpe) Strategy (sharpe_ratio)
   Created: Random Weight Strategy (dirichlet)
   Created: Black-Litterman Strategy (market_cap priors, upside views)


In [5]:
# Step 4: Set up backtesting
print("\n3. Setting up backtesting...")

backtester = Backtester(
    initial_capital=100000.0,
    risk_free_rate=0.03,
    transaction_costs=0.001
)

# Load data into backtester
backtester.load_data(data_loader, tickers=sectors)


3. Setting up backtesting...
Loaded data for 27 tickers
Date range: 2015-09-02 00:00:00 to 2025-09-01 00:00:00
Total observations: 2513
